## Train Xception Model to recognize the Intended Gender for a Watch

In [19]:

import os

import numpy as np
import pandas as pd

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dropout, Flatten, Dense,GlobalAveragePooling2D
from keras import applications
#from keras import backend as K
from keras import callbacks

In [20]:
! ls -R binary_gender_masked/ | wc -l

10546


In [21]:
# dimensions of our images.
img_width, img_height = 299, 299

# dimensions of our images.
img_width, img_height = 299, 299
n_features = 1

train_data_dir = 'binary_gender_masked/train/'
validation_data_dir = 'binary_gender_masked/test/'

available_train_files = len(os.listdir(train_data_dir + 'female/')) \
    + len(os.listdir(train_data_dir + 'male/'))
available_test_files = len(os.listdir(validation_data_dir + 'female/')) \
    + len(os.listdir(validation_data_dir+'male/'))
    
nb_train_samples  = available_train_files - available_train_files % 8
nb_validation_samples = available_test_files - available_test_files % 8

batch_size = 8

In [22]:
available_train_files % 8

0

Create Generators

In [23]:
datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=90,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=True,
    class_mode = 'binary')

test_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=True,
    class_mode = 'binary')

Found 7368 images belonging to 2 classes.
Found 3159 images belonging to 2 classes.


Load Standard Xception Model

In [24]:
model = applications.xception.Xception(include_top=False, weights='imagenet')

Build My "Custom" Model

In [25]:
watch_model = Sequential()
watch_model.add(model)
watch_model.add(GlobalAveragePooling2D(name='avg_pool'))
watch_model.add(Dense(2048, activation="relu"))
watch_model.add(Dropout(0.5))
watch_model.add(Dense(1, activation="sigmoid"))

In [26]:
watch_model.layers[0].summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, None, None, 32 864         input_2[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, None, None, 32 128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, None, None, 32 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [34]:
watch_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, None, None, 2048)  20861480  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 2049      
Total params: 25,059,881
Trainable params: 4,198,401
Non-trainable params: 20,861,480
_________________________________________________________________


Freeze Exception Layers

In [28]:
for layer in watch_model.layers[0].layers:
    layer.trainable = False

In [29]:
watch_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, None, None, 2048)  20861480  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 2049      
Total params: 25,059,881
Trainable params: 4,198,401
Non-trainable params: 20,861,480
_________________________________________________________________


Compile

In [32]:
watch_model.compile(
    loss = "binary_crossentropy", 
    optimizer='sgd', 
    metrics=["binary_accuracy"])

In [35]:
#Fit 

epochs = 5

cb1 = [callbacks.ModelCheckpoint(
    'xception_binary_gender_masked_best1.h5',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto', period=1)]

# fine-tune the model
watch_model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data = test_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks = cb1)

watch_model.save('xception_binary_gender_masked_final1.h5')

Epoch 1/5
921/921 [==============================] - 455s - loss: 0.4294 - binary_accuracy: 0.8008 - val_loss: 0.4083 - val_binary_accuracy: 0.8156
Epoch 2/5
921/921 [==============================] - 455s - loss: 0.4159 - binary_accuracy: 0.8134 - val_loss: 0.3734 - val_binary_accuracy: 0.8343
Epoch 3/5
921/921 [==============================] - 454s - loss: 0.4078 - binary_accuracy: 0.8142 - val_loss: 0.3773 - val_binary_accuracy: 0.8362
Epoch 4/5
921/921 [==============================] - 455s - loss: 0.4105 - binary_accuracy: 0.8104 - val_loss: 0.3626 - val_binary_accuracy: 0.8448
Epoch 5/5
921/921 [==============================] - 454s - loss: 0.4145 - binary_accuracy: 0.8097 - val_loss: 0.3849 - val_binary_accuracy: 0.8274


In [36]:
del watch_model

In [37]:
watch_model = load_model('xception_binary_gender_masked_final1.h5')

In [43]:
#Unfreeze a few more layers and allow to run

# watch_model.layers[0].summary()

# Freeze convolutional layers
for layer in watch_model.layers[0].layers[-6:]:
    layer.trainable = True
    
watch_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, None, None, 2048)  20861480  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 2049      
Total params: 25,059,881
Trainable params: 8,947,201
Non-trainable params: 16,112,680
_________________________________________________________________


In [44]:
epochs = 10


watch_model.compile(
    loss = "binary_crossentropy", 
    optimizer='sgd', 
    metrics=["binary_accuracy"])

# watch_model.layers[0].summary()

cb2 = [callbacks.ModelCheckpoint(
    'xception_binary_gender_masked_best2.h5',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto', period=1)]

# fine-tune the model
watch_model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data = test_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks = cb2)


watch_model.save('xception_binary_gender_masked_final2.h5')

Epoch 1/10
921/921 [==============================] - 480s - loss: 0.2561 - binary_accuracy: 0.8903 - val_loss: 0.2476 - val_binary_accuracy: 0.9000
Epoch 2/10
921/921 [==============================] - 479s - loss: 0.2354 - binary_accuracy: 0.9036 - val_loss: 0.2454 - val_binary_accuracy: 0.9013
Epoch 3/10
921/921 [==============================] - 478s - loss: 0.2256 - binary_accuracy: 0.9103 - val_loss: 0.2174 - val_binary_accuracy: 0.9194
Epoch 4/10
921/921 [==============================] - 478s - loss: 0.2197 - binary_accuracy: 0.9103 - val_loss: 0.2246 - val_binary_accuracy: 0.9102
Epoch 5/10
921/921 [==============================] - 478s - loss: 0.2001 - binary_accuracy: 0.9224 - val_loss: 0.2467 - val_binary_accuracy: 0.9073
Epoch 6/10
921/921 [==============================] - 479s - loss: 0.1973 - binary_accuracy: 0.9190 - val_loss: 0.2159 - val_binary_accuracy: 0.9159
Epoch 7/10
921/921 [==============================] - 479s - loss: 0.2030 - binary_accuracy: 0.9164 - val_

In [39]:
watch_model = load_model('xception_binary_gender_masked_final2.h5')

In [6]:
#Unfreeze a few more layers and allow to run

# watch_model.layers[0].summary()

# # Freeze convolutional layers
# for layer in watch_model.layers[0].layers[-3:]:
#     layer.trainable = True

epochs = 10


watch_model.compile(
    loss = "binary_crossentropy", 
    optimizer='sgd', 
    metrics=["binary_accuracy"])

watch_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, None, None, 2048)  20861480  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2049      
Total params: 20,863,529
Trainable params: 3,165,697
Non-trainable params: 17,697,832
_________________________________________________________________


In [ ]:
cb3 = [callbacks.ModelCheckpoint(
    'xception_binary_gender_masked_best3.h5',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto', period=1)]

# fine-tune the model
watch_model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data = test_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks = cb3)


watch_model.save('xception_binary_gender_masked_final3.h5')

Epoch 1/10
921/921 [==============================] - 467s - loss: 0.3190 - binary_accuracy: 0.8609 - val_loss: 0.2968 - val_binary_accuracy: 0.8788
Epoch 2/10
111/921 [==>...........................] - ETA: 276s - loss: 0.3422 - binary_accuracy: 0.8604